# Graphs and NetworkX

A graph is a data structure consisting of nodes (entities), and edges (relationships) which connect certain nodes to each other. Edges may have costs associated with them. NetworkX is a library that allows for computation over graphs.

NetworkX graph types you’ll use most:

- nx.Graph() → undirected - Edge from A to B is the same as edge from B to A.

- Not for this assignment: nx.DiGraph() → directed - Edge from A to B does not imply an edge from B to A.

You can see a full tutorial here: https://networkx.org/documentation/stable/tutorial.html

For this assignment, we will use the Graph class in graph.py; this lets you track additional information about your graph search.

In [ ]:
import networkx as nx
from graph_wrapper import Graph
import matplotlib.pyplot as plt
import math

%load_ext autoreload
%autoreload 2


## Example undirected friend graph
Alice is friends with Bob, Bob is friends with Carol. Damian has no friends.

In [ ]:
G = Graph()

G.add_node("Alice")
G.add_node("Bob")
G.add_node("Carol")
G.add_node("Damian")

# Alice is friends with Bob, Bob is friends with Carol. Damian has no friends.
G.add_edge("Alice", "Bob")
G.add_edge("Bob", "Carol")

print(G.nodes())
print(G.edges())

## Get node neighbors

I wrap the neighbors in list to make printing nice; but you can loop through without casting to list.

In [ ]:
print(list(G.neighbors("Alice")))
print(list(G.neighbors("Bob")))
print(list(G.neighbors("Damian")))
print('---')
# How to loop through them
for neighbor in G.neighbors("Bob"):
    print(neighbor)

In [ ]:
# You can draw NetworkX graphs using the matplotlib library
pos = nx.spring_layout(G)  # Options for layouts: https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.layout
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=800,
    font_size=10
)
plt.show()

# Graph with weights:

For this example, let's make a simple Manhattan-like grid, where the coordinates for each landmark are the (avenue, street). The edge weights will be the "manhattan" distance between the two connected landmarks: `|x2 - x1| + |y2 - y1|`. 

If you want to see how it is made, check premade_graphs.py

In [ ]:
from premade_graphs import basic_manhattan

manhattan_graph = basic_manhattan()

Basic Info

In [ ]:
print("Nodes:", manhattan_graph.number_of_nodes())
print("Edges:", manhattan_graph.number_of_edges())

Metadata for each node

In [ ]:
print("Nodes:", manhattan_graph.nodes(data=True))

print("Edges:", manhattan_graph.edges(data=True))


## Plot of this graph 
If you want to see how it is plotted, see utils.py

In [ ]:
from utils import draw_manhattan_graph

draw_manhattan_graph(manhattan_graph, title="NYC Manhattan Grid Landmark Graph (edge weight = blocks)")

## Examples of things to request

In [ ]:
# All neighbors of Union Square
print(list(manhattan_graph.neighbors("Union Square")))

# Get edge weight from Union Square to Chelsea
print(manhattan_graph.get_edge_data("Union Square", "Chelsea (23rd St)").get('weight'))

# Full attributes with Union Square node
print(manhattan_graph["Union Square"])

# Union Square position - works because this graph was given x, y coordinates
print(manhattan_graph.nodes["Union Square"]["pos"])


## Sample BFS function. Is that the best path though?

In [ ]:
def bfs(graph, start, goal, verbose=False):
    graph.reset_tracking()  # For tracking purposes
    
    # queue & previous path to get there
    queue = [
        (start, [start])
    ]

    while queue:  # While queue still has something to check
        cur_label, cur_path = queue.pop(0)  # Pop from end of queue for DFS
        # TODO: Avoid cycles
        if verbose: print(f"Checking {cur_label}, path={cur_path}")
        if cur_label == goal:
            return cur_path
    
        # Add neighbors to queue
        neighbors = graph.neighbors(cur_label)
        for neighbor in neighbors:
            queue.append(
                (neighbor, cur_path + [neighbor])
            )
        if verbose:
            print("Queue: ",  [x[0] for x in queue])
            print("-----")
    if verbose: print("No path found")
    return []

## BFS from Pace to Times Square

Turn verbose=True if you want to trace it

In [ ]:
path = bfs(manhattan_graph, "Pace", "Times Square", verbose=False)
cost = nx.path_weight(manhattan_graph, path, weight="weight")
print("Returned path length", len(path))
print("Returned path cost: ", cost)

In [ ]:
# Graphed out
draw_manhattan_graph(manhattan_graph, path=path, title="BFS on Simplified Manhattan")

In [ ]:
# With the Graph class I implemented, you can see statistics about your graph search
import pprint
pprint.pp(manhattan_graph.stats())

This is obviously duplicating a lot of work. Try adding detection for duplicates and see if that helps reduce the search space while also keeping the same path length.

And I'm sure you know by now, but this is not the most optimal path when considering edge costs.